# MIKH Model: Startup Shear (Stress Overshoot)

## Learning Objectives

1. Fit the **MIKH** model to startup shear data with **stress overshoot**
2. Understand the interplay of **kinematic hardening** and **thixotropy** in transient response
3. Analyze rate-dependent overshoot peak magnitude and timing
4. Use return mapping formulation for strain-driven protocols
5. Apply Bayesian inference to quantify parameter uncertainty

## Prerequisites

- NB01: MIKH Flow Curve (calibrates base parameters)

## Runtime

- Fast demo: ~3-4 minutes
- Full run: ~12-15 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.ikh import MIKH

# Add examples/utils to path for tutorial utilities
sys.path.insert(0, os.path.join("..", "utils"))
from ikh_tutorial_utils import (
    load_pnas_startup,
    save_ikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_mikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Startup Shear Response

When a constant shear rate $\dot{\gamma}$ is suddenly applied to a thixotropic yield-stress fluid, the stress response typically shows:

### Stress Overshoot Mechanism

1. **Initial elastic response**: $\sigma \approx G \gamma = G \dot{\gamma} t$ (linear increase)
2. **Yield and plasticity onset**: When $|\sigma - \alpha| > \sigma_y$, plastic flow begins
3. **Overshoot peak**: Competition between hardening and structure breakdown
4. **Steady state**: Balance of aging and rejuvenation

### Key Equations

**Return mapping algorithm** (incremental stress update):
$$
\sigma^{trial} = \sigma^n + G \Delta\gamma
$$
$$
\sigma^{n+1} = \sigma^{trial} - G \Delta\gamma^p \cdot \text{sign}(\sigma^{trial} - \alpha)
$$

**Kinematic hardening** (backstress evolution):
$$
\alpha^{n+1} = \alpha^n + C \Delta\gamma_p - \gamma_{dyn} |\alpha|^{m-1} \alpha |\Delta\gamma_p|
$$

**Structure evolution** (at each time step):
$$
\lambda^{n+1} = \lambda^n + \left[\frac{1-\lambda^n}{\tau_{thix}} - \Gamma \lambda^n |\dot{\gamma}^p|\right] \Delta t
$$

### Overshoot Physics

- **Kinematic hardening** ($C$, $\gamma_{dyn}$): Shifts yield surface, creates backstress
- **Structure breakdown** ($\Gamma$, $\tau_{thix}$): Reduces yield stress during flow
- **Peak position**: Scales with $1/\dot{\gamma}$ (strain-controlled)
- **Peak magnitude**: Increases with $\dot{\gamma}$ (rate-dependent)

## 3. Load Data

We load startup shear data from the PNAS Digital Rheometer Twin dataset.

In [ ]:
# Load startup data at different shear rates
rates = [0.056, 0.32, 1.0, 56.2, 100.0]
datasets = {}

for rate in rates:
    t, stress = load_pnas_startup(gamma_dot=rate)
    datasets[rate] = {"time": t, "stress": stress}
    print(f"gamma_dot = {rate:6.3f} 1/s: {len(t)} points, t_max = {t.max():.1f} s")

In [ ]:
# Plot all startup curves
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(rates)))

for i, rate in enumerate(rates):
    d = datasets[rate]
    ax.plot(d["time"], d["stress"], "-", color=colors[i], lw=1.5, 
            label=f"$\\dot{{\\gamma}}$ = {rate} 1/s")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Startup Shear: PNAS Digital Rheometer Twin Data", fontsize=13)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 5)  # Focus on overshoot region
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

### 4.1 Fit at Single Rate (gamma_dot = 1 1/s)

In [ ]:
# Select reference rate for primary fitting
ref_rate = 1.0
t_data = datasets[ref_rate]["time"]
stress_data = datasets[ref_rate]["stress"]

# Create and fit MIKH model
model = MIKH()
param_names = get_mikh_param_names()

t0 = time.time()
model.fit(t_data, stress_data, test_mode="startup", gamma_dot=ref_rate)
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Compute fit quality
stress_pred = model.predict_startup(t_data, gamma_dot=ref_rate)
metrics = compute_fit_quality(stress_data, stress_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g} Pa")

In [ ]:
# Plot fit
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t_data, stress_data, "ko", markersize=4, alpha=0.5, label="Data")
ax.plot(t_data, stress_pred, "-", lw=2.5, color="C0", label="MIKH fit")

# Mark overshoot peak
peak_idx = np.argmax(stress_pred)
ax.axvline(t_data[peak_idx], color="red", linestyle="--", alpha=0.5)
ax.annotate(f"Peak: {float(stress_pred[peak_idx]):.1f} Pa",
            xy=(t_data[peak_idx], stress_pred[peak_idx]),
            xytext=(t_data[peak_idx] + 0.2, stress_pred[peak_idx] + 2),
            fontsize=10, color="red")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title(f"MIKH Startup Fit ($\\dot{{\\gamma}}$ = {ref_rate} 1/s, R$^2$ = {metrics['R2']:.5f})", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, min(t_data.max(), 3))
plt.tight_layout()
display(fig)
plt.close(fig)

### 4.2 Rate-Dependent Analysis

In [ ]:
# Predict startup at all rates with calibrated model
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, rate in enumerate(rates[:5]):
    ax = axes[i]
    d = datasets[rate]
    t = d["time"]
    stress = d["stress"]
    
    # Model prediction at this rate
    stress_pred = model.predict_startup(t, gamma_dot=rate)
    
    ax.plot(t, stress, "ko", markersize=3, alpha=0.4, label="Data")
    ax.plot(t, stress_pred, "-", lw=2, color="C0", label="MIKH")
    
    ax.set_xlabel("Time [s]", fontsize=10)
    ax.set_ylabel("Stress [Pa]", fontsize=10)
    ax.set_title(f"$\\dot{{\\gamma}}$ = {rate} 1/s", fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, min(5, t.max()))
    if i == 0:
        ax.legend(fontsize=8)

# Remove empty subplot
axes[-1].axis("off")

plt.suptitle("Rate-Dependent Startup Response", fontsize=13)
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference with NUTS

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {name: model.parameters.get_value(name) for name in param_names}

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    t_data,
    stress_data,
    test_mode="startup",
    gamma_dot=ref_rate,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
all_pass = print_convergence_summary(result, param_names)

In [ ]:
# Trace plots for kinematic hardening parameters
idata = result.to_inference_data()
hardening_params = ["C", "gamma_dyn", "m", "sigma_y0", "delta_sigma_y"]
axes = az.plot_trace(idata, var_names=hardening_params, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Kinematic Hardening Parameters)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Posterior predictive check
posterior = result.posterior_samples

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t_data, stress_data, "ko", markersize=4, alpha=0.5, label="Data")

# Plot posterior samples
n_draws = min(50, len(list(posterior.values())[0]))
for i in range(n_draws):
    params_i = {name: posterior[name][i] for name in param_names}
    for name, val in params_i.items():
        model.parameters.set_value(name, float(val))
    pred_i = model.predict_startup(t_data, gamma_dot=ref_rate)
    ax.plot(t_data, pred_i, "-", color="C0", alpha=0.1, lw=0.5)

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Posterior Predictive Check (Startup)", fontsize=13)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, min(t_data.max(), 3))
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# NLSQ vs Bayesian comparison
print_parameter_comparison(model, posterior, param_names)

## 6. Physical Interpretation

### Stress Overshoot Mechanism

The overshoot arises from two competing mechanisms:

1. **Initial elastic loading**: Stress increases as $\sigma \approx G \dot{\gamma} t$
2. **Plastic yielding + structure breakdown**: Once yielded, stress decreases as:
   - Backstress $\alpha$ develops (kinematic hardening)
   - Structure $\lambda$ decreases (thixotropy)

### Rate Dependence

- **Peak strain** ($\gamma_{peak}$): Approximately constant (strain-controlled overshoot)
- **Peak stress** ($\sigma_{peak}$): Increases with $\dot{\gamma}$ (rate sensitivity)
- **Peak time** ($t_{peak}$): Scales as $\gamma_{peak}/\dot{\gamma}$

## 7. Save Results

In [ ]:
# Save results
save_ikh_results(model, result, "mikh", "startup", param_names)

## Key Takeaways

1. **Stress overshoot** in startup shear results from initial elastic loading followed by plastic flow and structure breakdown

2. **Kinematic hardening parameters** ($C$, $\gamma_{dyn}$) control the backstress evolution and overshoot shape

3. **Thixotropic parameters** ($\tau_{thix}$, $\Gamma$) determine how quickly structure breaks down during flow

4. **Rate dependence**: Higher $\dot{\gamma}$ leads to larger overshoot peak but shorter peak time

5. **Return mapping formulation** handles strain-driven protocols efficiently

### Next Steps

- **NB03**: Stress relaxation (synthetic from NB01 calibration)
- **NB04**: Creep response (real ML-IKH data)
- **NB05**: SAOS (synthetic from NB01 calibration)
- **NB06**: LAOS (real PNAS data)